# ChatGPT Playground

In [39]:
import openai as oai
import dotenv
import os

## system message for the assistant

You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction (e.g. right turn),  the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self intersecting. Return the road description in json format.

In [40]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# RoadGPT Assistant ID

asst_fqEGbKb98OI3nwpxrLQyfNYg

In [41]:
# import the openai api key

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = oai.OpenAI(api_key=OPENAI_API_KEY)

roadgpt = client.beta.assistants.create(
    name = "RoadGPT",
    instructions = """ You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you 
    split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction 
    (e.g. right turn), the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self 
    intersecting. Return the road description in json format of road_segment1, road_segment2, etc. """,
    model = "gpt-3.5-turbo"
)

show_json(roadgpt)

{'id': 'asst_R7E0KPj4jvmsAQQcvYZXjtuP',
 'created_at': 1709648649,
 'description': None,
 'file_ids': [],
 'instructions': " You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you \n    split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction \n    (e.g. right turn), the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self \n    intersecting. Return the road description in json format of road_segment1, road_segment2, etc. ",
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'RoadGPT',
 'object': 'assistant',
 'tools': []}

In [42]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_McKJb6KjbWDx2EukMTQjfSbR',
 'created_at': 1709648649,
 'metadata': {},
 'object': 'thread'}

In [43]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a mountain road with serpentines",
)

show_json(message)

{'id': 'msg_6AfX677f5vu8KOIOzm8Zrhda',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'Create a mountain road with serpentines'},
   'type': 'text'}],
 'created_at': 1709648649,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_McKJb6KjbWDx2EukMTQjfSbR'}

In [44]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=roadgpt.id,
)
show_json(run)

{'id': 'run_weSAufPqiZAszv85mGLQyrBS',
 'assistant_id': 'asst_R7E0KPj4jvmsAQQcvYZXjtuP',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1709648650,
 'expires_at': 1709649250,
 'failed_at': None,
 'file_ids': [],
 'instructions': " You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you \n    split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction \n    (e.g. right turn), the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self \n    intersecting. Return the road description in json format of road_segment1, road_segment2, etc. ",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_McKJb6KjbWDx2EukMTQjfSbR',
 'tool

In [45]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [46]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_weSAufPqiZAszv85mGLQyrBS',
 'assistant_id': 'asst_R7E0KPj4jvmsAQQcvYZXjtuP',
 'cancelled_at': None,
 'completed_at': 1709648655,
 'created_at': 1709648650,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': " You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you \n    split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction \n    (e.g. right turn), the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self \n    intersecting. Return the road description in json format of road_segment1, road_segment2, etc. ",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1709648650,
 'status': 'completed',
 'thread_id': 'thread_McKJb6KjbWDx2EukMTQjfSbR

In [47]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages.data[0].content[0].text.value

'{\n    "road_segment1": {\n        "length": 200,\n        "direction": "right turn",\n        "incline": 10,\n        "turn_degrees": 90\n    },\n    "road_segment2": {\n        "length": 150,\n        "direction": "straight",\n        "incline": 5\n    },\n    "road_segment3": {\n        "length": 100,\n        "direction": "left turn",\n        "incline": 10,\n        "turn_degrees": 120\n    },\n    "road_segment4": {\n        "length": 180,\n        "direction": "right turn",\n        "incline": 8,\n        "turn_degrees": 90\n    },\n    "road_segment5": {\n        "length": 150,\n        "direction": "straight",\n        "incline": 3\n    },\n    "road_segment6": {\n        "length": 120,\n        "direction": "left turn",\n        "incline": 12,\n        "turn_degrees": 100\n    }\n}'

# Rewrite the code as a class

In [55]:
class RoadGPT:

    def __init__(self):
        self.model = "gpt-3.5-turbo"
        self.instructions = """ You are a road designer. People give you descriptions of roads and you create more detailed descriptions of novel roads, where you 
        split the road into segments. Each turn / straight is it's own segment. For every segment return the length of the segment in meters, direction 
        (e.g. right turn), the incline in % andthe degrees of the turn in case it is a turn. The maximum incline is 15% and the road is not allowed to be self 
        intersecting. Return the road description in json format of road_segment1, road_segment2, etc. """
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        self.assistant_id = os.getenv("ROADGPT_ID")

        if not self.assistant_id:
            self.assistant_id = self.create_assistant("RoadGPT", self.instructions, self.model)

        self.threads = []
        self.runs = []


    def create_assistant(self, name, instructions, model):
        roadgpt = client.beta.assistants.create(
            name = name,
            instructions = instructions,
            model = model
        )

        return roadgpt.id
    
    
    def create_thread(self):
        thread = client.beta.threads.create()
        self.threads.append(thread.id)

        return thread.id
    

    def create_run(self, thread_id):
        run = client.beta.threads.runs.create(
            thread_id = thread_id,
            assistant_id = self.assistant_id
        )

        self.runs.append(run)

        return run
    

    def prompt(self, prompt, thread_id):
        assert thread_id, "You have to connect a thread."

        message = self.client.beta.threads.message.create(
            thread_id = thread_id,
            role = "user",
            content = prompt
        )

        




IndexError: list index out of range